Use [RISE](https://github.com/damianavila/RISE) to display the slides.

```bash
pip install RISE
```

<H2 style="text-align: center;">
Estonian Natural Language Toolkit
</H2>


<H1 style="text-align: center;">
Paul Tammo
</H1>

<H4 style="text-align: center;">
University of Tartu, STACC, Fujitsu
</H4>

<H4 style="text-align: center;">
PyCon Oct 3, 2019, Tallinn
</H4>

# Source

https://github.com/estnltk/estnltk/

# Installation
Optionally create and activate conda environment.
```bash
conda create -n estnltk python=3.6 -y
conda activate estnltk
```
Install EstNLTK 1.6 using precompiled packages in https://anaconda.org/estnltk/estnltk
```bash
conda install -c estnltk -c conda-forge estnltk
```

<center><img src='text_object.png'></center>

In [1]:
from estnltk import Text

text = Text('Mees oli kärbes.').analyse('all')
text

Text(text='Mees oli kärbes.')

In [2]:
text.morph_analysis

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Mees', [{'normalized_text': 'Mees', 'lemma': 'mees', 'root': 'mees', 'root_tokens': ['mees'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('oli', [{'normalized_text': 'oli', 'lemma': 'olema', 'root': 'ole', 'root_tokens': ['ole'], 'ending': 'i', 'clitic': '', 'form': 's', 'partofspeech': 'V'}]),
Span('kärbes', [{'normalized_text': 'kärbes', 'lemma': 'kärbes', 'root': 'kärbes', 'root_tokens': ['kärbes'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

# Taggers
## GrammarTagger

# PostgreSQL integration

# Project manager
Sven Laur

# Developers

Siim Orasmaa,
Timo Petmanson,
Uku Raudvere,
Dage Särg,
Paul Tammo,
Aleksandr Tkatšenko

# Consulting
Heiki-Jaan Kaalep,
Kadri Muschinek,
Kairit Sirts,
Tarmo Vaino